# Federated Protein Embeddings and Task Model Fitting with BioNeMo

<div class="alert alert-block alert-info"> <b>NOTE</b> This notebook was tested on a single A1000 GPU and is compatible with BioNeMo Framework v2.5. To leverage additional or higher-performance GPUs, you can modify the configuration files and simulation script to accommodate multiple devices and increase thread utilization respectively. </div>

This example notebook shows how to obtain protein learned representations in the form of embeddings using the ESM-1nv pre-trained model in a federated learning (FL) setting. The model is trained with NVIDIA's BioNeMo framework for Large Language Model training and inference. For more details, please visit NVIDIA BioNeMo Service at https://www.nvidia.com/en-us/gpu-cloud/bionemo.

This example is based on NVIDIA BioNeMo Service [example](https://github.com/NVIDIA/BioNeMo/blob/main/examples/service/notebooks/task-fitting-predictor.ipynb) 
but runs inference locally (on the FL clients) instead of using BioNeMo's cloud API.

This notebook will walk you through the task fitting workflow in the following sections:

* Dataset sourcing & Data splitting
* Federated embedding extraction
* Training an MLP to predict subcellular location

## Setup

<div class="alert alert-block alert-info"> <b>NOTE:</b> This notebook is designed to run inside the BioNeMo Framework Docker container. Follow these [instructions](https://docs.nvidia.com/ai-enterprise/deployment/vmware/latest/docker.html) to set up your Docker environment and execute the following bash script before opening this notebook.</div>

To set up your environment, simply run (outside this notebook):

```bash
./start_bionemo.sh
```

This script will automatically pull the [BioNeMo Docker container](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/clara/containers/bionemo-framework) (tested with version nvcr.io/nvidia/clara/bionemo-framework:2.5) and launch Jupyter Lab at `http://<your-hostname>:8888`. Open that URL in your browser and access this notebook.

For detailed setup guidance, refer to the [BioNeMo User Guide](https://docs.nvidia.com/bionemo-framework/latest/user-guide/).

Once you open this notebook, continue executing the cells below.

<div class="alert alert-block alert-info"> <b>NOTE:</b> Some cells below produce long outputs. To suppress them, we use:<br><br> <pre>%%capture --no-display --no-stderr cell_output</pre><br> Comment or remove this line to restore full output.</div>

### Import and install all required packages

In [ ]:
%%capture --no-display --no-stderr cell_output
! pip install "nvflare~=2.7.0rc"
! pip install biopython --no-dependencies

import io
import numpy as np
import os
import pickle
import re
import requests
import split_data

from Bio import SeqIO
from nvflare import SimulatorRunner  
from split_data import split

### Obtaining the protein embeddings using the BioNeMo ESM-1nv model
Using BioNeMo, each FL client can obtain numerical vector representations of protein sequences called embeddings. Protein embeddings can then be used for visualization or making downstream predictions.

Here we are interested in training a neural network to predict subcellular location from an embedding.

The data we will be using comes from the paper [Light attention predicts protein location from the language of life](https://academic.oup.com/bioinformaticsadvances/article/1/1/vbab035/6432029) by Stärk et al. In this paper, the authors developed a machine learning algorithm to predict the subcellular location of proteins from sequence through protein langage models that are similar to those hosted by BioNeMo. Protein subcellular location refers to where the protein localizes in the cell, for example a protein may be expressed in the Nucleus or in the Cytoplasm. Knowing where proteins localize can provide insights into the underlying mechanisms of cellular processes and help identify potential targets for drug development. The following image includes a few examples of subcellular locations in an animal cell:


(Image freely available at https://pixabay.com/images/id-48542)

### Dataset sourcing
For our target input sequences, we will point to FASTA sequences in a benchmark dataset called Fitness Landscape Inference for Proteins (FLIP). FLIP encompasses experimental data across adeno-associated virus stability for gene therapy, protein domain B1 stability and immunoglobulin binding, and thermostability from multiple protein families.

In [ ]:
# Example protein dataset location
fasta_url= "http://data.bioembeddings.com/public/FLIP/fasta/scl/mixed_soft.fasta"

First, we define the source of example protein dataset with the FASTA sequences. This data follows the [biotrainer](https://github.com/sacdallago/biotrainer/blob/main/docs/data_standardization.md) standard, so it includes information about the class in the FASTA header, and the protein sequence. Here are two example sequences in this file:

```
>Sequence1 TARGET=Cell_membrane SET=train VALIDATION=False
MMKTLSSGNCTLNVPAKNSYRMVVLGASRVGKSSIVSRFLNGRFEDQYTPTIEDFHRKVYNIHGDMYQLDILDTSGNHPFPAM
RRLSILTGDVFILVFSLDSRESFDEVKRLQKQILEVKSCLKNKTKEAAELPMVICGNKNDHSELCRQVPAMEAELLVSGDENC
AYFEVSAKKNTNVNEMFYVLFSMAKLPHEMSPALHHKISVQYGDAFHPRPFCMRRTKVAGAYGMVSPFARRPSVNSDLKYIKA
KVLREGQARERDKCSIQ
>Sequence4833 TARGET=Nucleus SET=train VALIDATION=False
MARTKQTARKSTGGKAPRKQLATKAARKSAPATGGVKKPHRFRPGTVALREIRKYQKSTELLIRKLPFQRLVREIAQDFKTDL
RFQSSAVAALQEAAEAYLVGLFEDTNLCAIHAKRVTIMPKDIQLARRIRGERA
Note the following attributes in the FASTA header:
```

* `TARGET` attribute holds the subcellular location classification for the sequence, for instance Cell_membrane and Nucleus. This dataset includes a total of ten subcellelular location classes -- more on that below.
* `SET` attribute defines whether the sequence should be used for training (train) or testing (test)
* `VALIDATION` attribute defines whether the sequence should be used for validation (all sequences where this is True are also in set=train)

### Downloading the protein sequences and subcellular location annotations
In this step we download the FASTA file defined above and parse the sequences into a list of BioPython SeqRecord objects.



In [ ]:
# Download the FASTA file from FLIP: https://github.com/J-SNACKKB/FLIP/tree/main/splits/scl
fasta_content = requests.get(fasta_url, headers={
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x86)'
}).content.decode('utf-8')
fasta_stream = io.StringIO(fasta_content)

# Obtain a list of SeqRecords/proteins which contain sequence and attributes
# from the FASTA header
proteins = list(SeqIO.parse(fasta_stream, "fasta"))
print(f"Downloaded {len(proteins)} sequences")

### Download Model Checkpoints

The following code will download the pre-trained model, `esm2n/8m:2.0`, from the NGC registry:

In [ ]:
from bionemo.core.data.load import load

checkpoint_path = load("esm2/8m:2.0")
# checkpoint_path = load("esm2/650m:2.0") for ESM2-650m and full experiments
print(checkpoint_path)

### Data splitting
Next, we prepare the data for simulating federated learning using `n_clients`. Note that a copy of the same test set is shared between the clients in this example.

In [ ]:
n_clients = 3
# limiting to the proteins with sequence length<512 for embedding queries
MAX_SEQUENCE_LEN = 512
seed=42
data_root = "/tmp/data/mixed_soft"
split_alpha = 1.0  # moderate label heterogeneity of alpha=1.0

np.random.seed(seed)

# Extract meta data and split
data = []
for i, x in enumerate(proteins):
        if len(str(x.seq)) > MAX_SEQUENCE_LEN:
            continue
            
        entry = {key: value for key, value in re.findall(r"([A-Z_]+)=(-?[A-z0-9]+[.0-9]*)", x.description)}
        entry["sequences"] = str(x.seq)
        entry["id"] = str(i)
        entry["labels"] = entry["TARGET"]
       
        data.append(entry)
print(f"Read {len(data)} valid sequences.")
               
# Split the data and save for each client
# Note, test_data is kept the same on each client and is not split
# `concat=False` is used for SCL experiments (see ../downstream/scl)
split(proteins=data, num_sites=n_clients, split_dir=data_root, alpha=split_alpha, concat=False)  
# `concat=True` is used for separate inference + MLP classifier in this notebook
split(proteins=data, num_sites=n_clients, split_dir=data_root, alpha=split_alpha, concat=True)  

### Federated embedding extraction
Here, we run inference on each client sequentially using one thread to preserve GPU memory. Running inference of the ESM-1nv model to extract embeddings with a micro-batch size of 64 takes about 30 GB GPU memory. You can reduce the micro-batch size if you have less GPU memory available. For more inference requirements, see the [model overview](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/clara/models/esm2nv650m). Note that this can take a while.

In [ ]:
from nvflare.app_opt.pt.recipes.fedavg import FedAvgRecipe
from nvflare.recipe import SimEnv

results_path = "/tmp/data/mixed_soft/results"

micro_bs = 64

# Create results directory
os.makedirs(results_path, exist_ok=True)

# Build script arguments for inference (same for all clients, paths resolved in client script)
script_args = f"--checkpoint-path {checkpoint_path} --data-root {data_root} --results-path {results_path} --precision bf16-mixed --micro-batch-size {micro_bs} --num-gpus 1"

# Create FedAvgRecipe for inference (1 round only as this is just inference)
recipe = FedAvgRecipe(
    name="esm2_embeddings",
    min_clients=n_clients,
    num_rounds=1,  # Inference only needs 1 round
    train_script="job_inference/client.py",
    train_args=script_args
)

# Run simulation
env = SimEnv(num_clients=n_clients, workspace_root="/tmp/nvflare/bionemo/embeddings", gpu_config="0", num_threads=1)
run = recipe.execute(env)
print()
print("Job Status:", run.get_status())
print("Results:", run.get_result())
print()

### Inspecting the embeddings and labels
Embeddings returned from the BioNeMo model are vectors of fixed size for each input sequence. In other words, if we input 10 sequences, we will obtain a matrix `10xD`, where `D` is the size of the embedding (in the case of ESM2-8m, `D=320` & for ESM2-650m, `D=1280`). At a glance, these real-valued vector embeddings don't show any obvious features (see the printout in the next cell). But these vectors do contain information that can be used in downstream models to reveal properties of the protein, for example the subcellular location as we'll explore below.

In [ ]:
# load embeddings from site-1
import torch

client_name = "site-1"
results = torch.load(f"{results_path}/inference_results_{client_name}/predictions__rank_0.pt")

for key, val in results.items():
    if val is not None:
        print(f'{key}\t{val.shape}')

protein_embeddings = results['embeddings']
print(f"Loaded {len(protein_embeddings)} embeddings from site-1.")

for i in range(4):
    x = protein_embeddings[i]
    print(f"{i+1}. embedding: range {x.min():.2f}-{x.max():.2f}, mean={x.mean():.2f}, shape={x.shape}")

Let's enumerate the labels corresponding to potential subcellular locations.

In [ ]:
# Let's also print all the labels

labels = set([entry['labels'] for entry in data])

for i, label in enumerate(labels):
    print(f"{i+1}. {label.replace('_', ' ')}")

### Training a MLP to predict subcellular location
To be able to classify proteins for their subcellular location, we train a simple scikit-learn Multi-layer Perceptron (MPL) classifier using Federated Averaging ([FedAvg](https://arxiv.org/abs/1602.05629)). The MLP model uses a network of hidden layers to fit the input embedding vectors to the model classes (the cellular locations above). In the simulation below, we define the MLP to use the Adam optimizer with a network of (512, 256, 128) hidden layers, defining a random state (or seed) for reproducibility, and trained for 30 rounds of FedAvg (see [config_fed_server.json](./jobs/fedavg/app/config/config_fed_server.json)). 

We can use the same configuration also to simulate local training where each client is only training with their own data by setting `os.environ["SIM_LOCAL"] = "True"`. Our [BioNeMoMLPLearner](./jobs/fedavg/app/custom/bionemo_mlp_learner.py) will then ignore the global weights coming from the server.

### Local training

In [ ]:
from nvflare.app_opt.pt.recipes.fedavg import FedAvgRecipe
from nvflare.recipe import SimEnv
from nvflare.recipe.utils import add_experiment_tracking

os.environ["SIM_LOCAL"] = "True"

embedding_dimensions = 320  # embedding dimensions of ESM2-8m

# Build script arguments for MLP training
script_args = f"--data-root {data_root} --results-path {results_path} --aggregation-epochs 4 --lr 1e-5 --batch-size 128 --embedding-dimensions {embedding_dimensions}"

# Create FedAvgRecipe for MLP training (local simulation)
recipe = FedAvgRecipe(
    name="mlp_local",
    min_clients=n_clients,
    num_rounds=100,
    train_script="job_fedavg/client.py",
    train_args=script_args
)

# Enable TensorBoard tracking
add_experiment_tracking(recipe, "tensorboard")

# Run simulation
env = SimEnv(
    num_clients=n_clients,
    workspace_root=f"/tmp/nvflare/bionemo/local_alpha{split_alpha}",
    num_threads=n_clients  # MLP is lightweight, can run in parallel
)
run = recipe.execute(env)
print()
print("Job Status:", run.get_status())
print("Results:", run.get_result())
print()


### Federated learning

In [ ]:
os.environ["SIM_LOCAL"] = "False"

embedding_dimensions = 320  # embedding dimensions of ESM2-8m

# Build script arguments for MLP training
script_args = f"--data-root {data_root} --results-path {results_path} --aggregation-epochs 4 --lr 1e-5 --batch-size 128 --embedding-dimensions {embedding_dimensions}"

# Create FedAvgRecipe for MLP training (federated)
recipe = FedAvgRecipe(
    name="mlp_fedavg",
    min_clients=n_clients,
    num_rounds=100,
    train_script="job_fedavg/client.py",
    train_args=script_args
)

# Enable TensorBoard tracking
add_experiment_tracking(recipe, "tensorboard")

# Run simulation
env = SimEnv(
    num_clients=n_clients,
    workspace_root=f"/tmp/nvflare/bionemo/fedavg_alpha{split_alpha}",
    num_threads=n_clients  # MLP is lightweight, can run in parallel
)
run = recipe.execute(env)
print()
print("Job Status:", run.get_status())
print("Results:", run.get_result())
print()


### TensorBoard Visualization
You can visualize the training progress using TensorBoard
```
tensorboard --logdir /tmp/nvflare/bionemo
```

Note, as the test set is shared between the clients in this example, we will see the same performance metrics for the global model computed on each client.

An example of local (green, blue, red) vs. federated (orange) training is shown below using the **ESM2-650m** model for inference. Note, the default model used in this notebook is ESM2-8m. You need to change the `checkpoint_path` and `embedding_dimensions` above when switching to a different model.

![TensorBoard training curves](tb_curve.png)